<a href="https://colab.research.google.com/github/chaturvediosmio/MNIST-Fashion-Classification/blob/main/MNIST_Fashion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [296]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as pl
import tensorflow as tf
from google.colab.patches import cv2_imshow
from keras.datasets import fashion_mnist
from keras.models import Sequential, save_model, load_model
from keras.layers import Conv2D, MaxPool2D, ReLU, Flatten, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical, load_img, img_to_array
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from PIL import Image

In [187]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

In [188]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [189]:
pd.DataFrame(y_train).value_counts()

9    6000
8    6000
7    6000
6    6000
5    6000
4    6000
3    6000
2    6000
1    6000
0    6000
dtype: int64

In [229]:
pd.DataFrame(y_test).value_counts()

9    1000
8    1000
7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
1    1000
0    1000
dtype: int64

In [231]:
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [191]:
X_train_flat = np.reshape(X_train, (60000, -1))
X_test_flat = np.reshape(X_test, (10000, -1))
print(X_train_flat.shape)
print(X_test_flat.shape)

(60000, 784)
(10000, 784)


In [192]:
df_train = pd.concat([pd.DataFrame(X_train_flat), pd.DataFrame(y_train, columns=['target'])], axis = 1)
df_test = pd.concat([pd.DataFrame(X_test_flat), pd.DataFrame(y_test, columns=['target'])], axis = 1)

cv2_imshow(np.reshape(df_train[df_train.target==0].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Tshirt
cv2_imshow(np.reshape(df_train[df_train.target==1].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Trousers
cv2_imshow(np.reshape(df_train[df_train.target==2].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Sweater
cv2_imshow(np.reshape(df_train[df_train.target==3].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Dress
cv2_imshow(np.reshape(df_train[df_train.target==4].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Jacket
cv2_imshow(np.reshape(df_train[df_train.target==5].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Sandals
cv2_imshow(np.reshape(df_train[df_train.target==6].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Shirt
cv2_imshow(np.reshape(df_train[df_train.target==7].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Sneakers
cv2_imshow(np.reshape(df_train[df_train.target==8].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Bag
cv2_imshow(np.reshape(df_train[df_train.target==9].sample(1).drop('target', axis = 1).to_numpy(), (28, 28))) #Boots

In [193]:
dict = {0:'Tshirt', 1:'Trousers', 2:'Sweater', 3:'Dress', 4:'Jacket', 5:'Sandals', 6:'Shirt', 7:'Sneakers', 8:'Bag', 9:'Boots' }

In [227]:
'''one hot encoded targets'''
y_train_hot = to_categorical(y_train) 
y_test_hot = to_categorical(y_test)

In [309]:
df_train.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,41.0,188.0,103.0,54.0,48.0,43.0,87.0,168.0,133.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,49.0,136.0,219.0,216.0,228.0,...,240.0,243.0,214.0,224.0,162.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,139.0,146.0,130.0,135.0,135.0,137.0,125.0,124.0,125.0,121.0,119.0,114.0,130.0,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,118.0,24.0,0.0,0.0,0.0,0.0,0.0,48.0,88.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,100.0,212.0,205.0,...,102.0,75.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,18.0,42.0,57.0,56.0,32.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.0,96.0,175.0,156.0,64.0,14.0,54.0,137.0,204.0,194.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0,186.0,177.0,183.0,175.0,188.0,...,152.0,85.0,160.0,133.0,100.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,35.0,4.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,189.0,206.0,187.0,32.0,...,242.0,255.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0,102.0,168.0,25.0,0.0,139.0,161.0,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [310]:
df_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,67.0,0.0,0.0,0.0,0.0,50.0,38.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,120.0,209.0,226.0,247.0,...,0.0,0.0,0.0,0.0,39.0,249.0,255.0,123.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,205.0,197.0,11.0,0.0,4.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,0.0,3.0,174.0,189.0,67.0,0.0,0.0,0.0,2
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,67.0,177.0,129.0,153.0,117.0,129.0,146.0,141.0,175.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,196.0,255.0,...,255.0,252.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.0,159.0,71.0,0.0,0.0,68.0,176.0,164.0,58.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.0,123.0,108.0,99.0,99.0,84.0,83.0,86.0,92.0,70.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106.0,248.0,230.0,...,248.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,174.0,101.0,0.0,0.0,0.0,146.0,146.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,57.0,67.0,73.0,76.0,76.0,83.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,13.0,79.0,128.0,201.0,...,174.0,197.0,182.0,162.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,72.0,95.0,109.0,111.0,111.0,106.0,101.0,71.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6


In [202]:
df_X_train_norm = pd.DataFrame(df_train.iloc[:, :-1].to_numpy()/255.0)
df_X_test_norm = pd.DataFrame(df_test.iloc[:, :-1].to_numpy()/255.0)

arr_X_train_norm = np.reshape( df_X_train_norm.to_numpy() , (-1, 28, 28, 1))
arr_X_test_norm = np.reshape( df_X_test_norm.to_numpy() , (-1, 28, 28, 1))

print(arr_X_train_norm.shape,'\n',arr_X_test_norm.shape)

(60000, 28, 28, 1) 
 (10000, 28, 28, 1)


In [247]:
df_X_test_norm

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.050980,0.262745,0.000000,0.000000,0.000000,0.000000,0.196078,0.149020,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.031373,0.470588,0.819608,0.886275,0.968627,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.152941,0.976471,1.0,0.482353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.192157,0.803922,0.772549,0.043137,0.0,0.015686,0.003922,0.007843,0.007843,0.007843,0.007843,0.007843,0.007843,0.007843,0.007843,0.011765,0.0,0.011765,0.682353,0.741176,0.262745,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.003922,0.000000,0.262745,0.694118,0.505882,0.600000,0.458824,0.505882,0.572549,0.552941,0.686275,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.007843,0.000000,0.768627,1.000000,...,0.631373,1.000000,0.988235,0.466667,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.384314,0.623529,0.278431,0.000000,0.000000,0.266667,0.690196,0.643137,0.227451,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.082353,0.482353,0.423529,0.388235,0.388235,0.329412,0.325490,0.337255,0.360784,0.274510,0.023529,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.415686,0.972549,0.901961,...,0.878431,0.972549,0.180392,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.078431,0.682353,0.396078,0.000000,0.000000,0.000000,0.572549,0.572549,0.003922,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.007843,0.0,0.003922,0.003922,0.0,0.000000,0.000000,0.000000,0.223529,0.262745,0.286275,0.298039,0.298039,0.325490,0.243137,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003922,0.003922,0.003922,0.0,0.000000,0.050980,0.309804,0.501961,0.788235,...,0.611765,0.682353,0.772549,0.713726,0.635294,0.239216,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.117647,0.282353,0.372549,0.427451,0.435294,0.435294,0.415686,0.396078,0.278431,0.047059,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [308]:
def build_model():
  model = Sequential()
  model.add(Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='he_normal', input_shape = (28,28,1)))
  model.add(MaxPool2D((2,2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer=Adam(learning_rate=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
  return(model)

In [251]:
def kfold_valid(X_train, y_train):
  kfold = KFold(n_splits=5, random_state=42, shuffle=True)
  score=[]
  hist = []
  for train_i, test_i in kfold.split(X_train, y_train):
    model = build_model()
    X_tr, y_tr, X_val, y_val = X_train[train_i], y_train[train_i], X_train[test_i], y_train[test_i]
    callback = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.002)
    history = model.fit(X_tr, y_tr, batch_size=32, epochs=20, validation_data=(X_val, y_val), callbacks=[callback])
    loss_val, accuracy = model.evaluate(X_val, y_val)
    print('> %.3f' % (accuracy * 100.0))
    score.append(accuracy)
    hist.append(history)
  model.save('final_model.h5')
  return (score, hist)

In [252]:
scores, histories = kfold_valid(arr_X_train_norm, y_train_hot)

Epoch 1/20
1500/1500 [==============================] - 29s 19ms/step - loss: 0.3964 - accuracy: 0.8602 - val_loss: 0.2984 - val_accuracy: 0.8907
Epoch 2/20
1500/1500 [==============================] - 29s 19ms/step - loss: 0.2572 - accuracy: 0.9063 - val_loss: 0.2844 - val_accuracy: 0.8967
Epoch 3/20
1500/1500 [==============================] - 29s 19ms/step - loss: 0.2126 - accuracy: 0.9213 - val_loss: 0.2567 - val_accuracy: 0.9074
Epoch 4/20
1500/1500 [==============================] - 29s 20ms/step - loss: 0.1785 - accuracy: 0.9350 - val_loss: 0.2659 - val_accuracy: 0.9068
Epoch 5/20
1500/1500 [==============================] - 30s 20ms/step - loss: 0.1511 - accuracy: 0.9437 - val_loss: 0.2678 - val_accuracy: 0.9101
Epoch 6/20
375/375 [==============================] - 2s 6ms/step - loss: 0.2912 - accuracy: 0.9078
> 90.783
Epoch 1/20
1500/1500 [==============================] - 31s 20ms/step - loss: 0.4062 - accuracy: 0.8578 - val_loss: 0.2982 - val_accuracy: 0.8945
Epoch 2/20
1500

313/313 [==============================] - 2s 6ms/step - loss: 2.6984 - accuracy: 0.1208
> 12.080


In [253]:
model_load = load_model('final_model.h5')
loss, acc = model_load.evaluate(arr_X_test_norm, y_test_hot)
print('> %.3f' % (acc * 100.0))

313/313 [==============================] - 2s 6ms/step - loss: 0.3267 - accuracy: 0.9053
> 90.530


In [304]:
path = 'blank_tshirt.jpg'
img_orig = load_img(path, color_mode = 'grayscale', target_size=(28,28))
img_arr = img_to_array(img_orig).reshape(1,28,28,1)
img = img_arr.astype('float32')
img /= 255.0
print(img.shape)


(1, 28, 28, 1)


In [307]:
path = 'blank_tshirt.jpg'
img_orig = load_img(path, color_mode = 'grayscale', target_size=(28,28))
img_arr = img_to_array(img_orig)
cv2_imshow(img_arr)

In [306]:
result = model_load.predict(img)
print(dict[np.argmax(result[0])])

Tshirt
